In [1]:
import pandas as pd

# ITR

In [3]:
path = '../data/processed/metrics_raw_data.csv'

metrics_raw = pd.read_csv(path)
metrics_raw.head()

,Date,Year,Q,Year_Quarter,Product_ID,Warehouse_Loc_ID,Daily_Sales,Stock_Level,Inventory_Holding_Cost,Lost_Sales
0,2023-03-28,2023,1,2023-Q1,PROD1000,Surabaya-WH1,48,114,37717.78,2
1,2023-01-21,2023,1,2023-Q1,PROD1001,Surabaya-WH1,16,327,11276.30,4
2,2023-02-24,2023,1,2023-Q1,PROD1002,Surabaya-WH1,22,378,25660.33,0
3,2023-01-17,2023,1,2023-Q1,PROD1003,Surabaya-WH1,2,120,23731.02,5
4,2023-03-02,2023,1,2023-Q1,PROD1003,Surabaya-WH1,9,315,2011.11,1


In [11]:
metrics_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958040 entries, 0 to 1958039
Data columns (total 10 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Date                    object 
 1   Year                    int64  
 2   Q                       int64  
 3   Year_Quarter            object 
 4   Product_ID              object 
 5   Warehouse_Loc_ID        object 
 6   Daily_Sales             int64  
 7   Stock_Level             int64  
 8   Inventory_Holding_Cost  float64
 9   Lost_Sales              int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 149.4+ MB


In [23]:
metrics_raw['Date'] = pd.to_datetime(metrics_raw['Date'])
itr = metrics_raw.groupby('Year_Quarter', as_index=False).agg({'Date':['min','max'], 'Daily_Sales':'sum'})
itr.columns = ['_'.join(col).strip() if col[1] else col[0] for col in itr.columns.values]

metrics_raw[metrics_raw['Date']=='2023-01-01']['Stock_Level'].sum()

592351

In [27]:
metrics_raw['Date'] = pd.to_datetime(metrics_raw['Date'])
itr = metrics_raw.groupby('Year_Quarter', as_index=False).agg({'Date':['min','max'], 'Daily_Sales':'sum'})
itr.columns = ['_'.join(col).strip() if col[1] else col[0] for col in itr.columns.values]

earliest_stock_sum = [(metrics_raw[metrics_raw['Date']==i]['Stock_Level'].sum()) for i in itr['Date_min']]
latest_stock_sum = [(metrics_raw[metrics_raw['Date']==i]['Stock_Level'].sum()) for i in itr['Date_max']]

itr['Earliest_Stock_Sum'] = earliest_stock_sum
itr['Latest_Stock_Sum'] = latest_stock_sum

itr['ITR'] = (itr['Daily_Sales_sum']/((itr['Latest_Stock_Sum']+itr['Earliest_Stock_Sum'])/2)).round(2)
itr.drop(columns=['Date_min','Date_max', 'Daily_Sales_sum', 'Earliest_Stock_Sum', 'Latest_Stock_Sum'], inplace=True)
itr


,Year_Quarter,ITR
0,2023-Q1,8.94
1,2023-Q2,9.06
2,2023-Q3,9.11
3,2023-Q4,9.26
4,2024-Q1,9.09
5,2024-Q2,9.05
6,2024-Q3,9.20
7,2024-Q4,5.11


# Overstock Cost

In [10]:
overstock_cost = metrics_raw.copy()
overstock_cost['Overstock_Cost'] = (overstock_cost['Stock_Level'] - overstock_cost['Daily_Sales']) * overstock_cost['Inventory_Holding_Cost']
overstock_cost = overstock_cost.groupby('Year_Quarter', as_index=False).agg({'Overstock_Cost':'sum'})
# overstock_cost['Overstock_Cost'] = overstock_cost['Overstock_Cost'].apply(lambda x: f'{x:,.0f}')
overstock_cost

,Year_Quarter,Overstock_Cost
0,2023-Q1,"1,254,644,548,003"
1,2023-Q2,"1,360,983,074,985"
2,2023-Q3,"1,468,744,297,523"
3,2023-Q4,"1,563,828,971,583"
4,2024-Q1,"1,565,278,789,748"
5,2024-Q2,"1,566,541,531,374"
6,2024-Q3,"1,590,625,517,500"
7,2024-Q4,"875,961,544,471"


# Stockout Ratio

In [11]:
metrics_raw.head()

,Date,Year,Q,Year_Quarter,Product_ID,Warehouse_Loc_ID,Daily_Sales,Stock_Level,Inventory_Holding_Cost,Lost_Sales
0,2023-03-28,2023,1,2023-Q1,PROD1000,Surabaya-WH1,48,114,37717.78,2
1,2023-01-21,2023,1,2023-Q1,PROD1001,Surabaya-WH1,16,327,11276.30,4
2,2023-02-24,2023,1,2023-Q1,PROD1002,Surabaya-WH1,22,378,25660.33,0
3,2023-01-17,2023,1,2023-Q1,PROD1003,Surabaya-WH1,2,120,23731.02,5
4,2023-03-02,2023,1,2023-Q1,PROD1003,Surabaya-WH1,9,315,2011.11,1


In [1]:
a = [1,2,3]
b = None

c = a or b

print(c)

[1, 2, 3]


In [6]:
a = list(set([1,2,3]).union(set([2,3,4])))
print(a)

[1, 2, 3, 4]
